In [1]:
!pip install mlflow pyvi minio -q
!pip install hf_xet -q

import mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 52.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 80.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.5/720.5 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
mlflow.set_tracking_uri("http://36.50.135.226:7893/")

# from mlflow.tracking import MlflowClient

# client = MlflowClient()
# client.restore_experiment(experiment_id="2")

mlflow.set_experiment(experiment_id="10")

<Experiment: artifact_location='s3://mlflow/6', creation_time=1745743711248, experiment_id='10', last_update_time=1745743711248, lifecycle_stage='active', name='ImageCaption_TPC37k_BartPho-ViT-GPT2_LoRALayerFT', tags={}>

In [3]:
import os

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://36.50.135.226:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"

# MinIO

In [4]:
from minio import Minio
from minio.error import S3Error
import glob

# Training

In [5]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import transformers
from transformers import VisionEncoderDecoderModel, ViTImageProcessor
import torch
from PIL import Image
import torch.nn as nn
import cv2
import torchvision
from tqdm.notebook import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import ViTForImageClassification, ViTImageProcessor
from collections import OrderedDict
from transformers import GPT2Config, GPT2LMHeadModel
import mlflow
from mlflow.models import infer_signature
import mlflow.pytorch
import re
from pyvi import ViTokenizer
from torch.optim import AdamW
from datetime import datetime
import json
import pickle
from io import BytesIO

2025-05-26 15:37:10.723213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748273831.138258      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748273831.277461      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 1. Chuẩn bị file thư viện

In [6]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
with open("requirements.txt") as f:
    pip_reqs = [line.strip() for line in f if line.strip()]

# 2. Load data

## API Data

In [15]:
# 1. Gửi GET request đến API metadata
url = "http://36.50.135.226/api/v1/metadata/encoded-data"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

# 2. Kiểm tra phản hồi
if response.status_code == 200:
    data = response.json()
    object_keys = data.get("object_keys", [])

    def extract_timestamp(key):
        match = re.search(r"encoded_data_(\d{14})\.pkl", key)
        if match:
            return datetime.strptime(match.group(1), "%Y%m%d%H%M%S")
        return None

    if object_keys:
        base_url = "http://160.191.244.13:9000/lakehouse/"
        all_pkl_data = []  # Biến chứa toàn bộ dữ liệu

        for key in sorted(object_keys, key=extract_timestamp):
            file_url = base_url + key
            file_response = requests.get(file_url)

            if file_response.status_code == 200:
                try:
                    file_bytes = BytesIO(file_response.content)
                    pkl_data = pickle.load(file_bytes)

                    # Gộp dữ liệu
                    if isinstance(pkl_data, list):
                        all_pkl_data.extend(pkl_data)
                    elif isinstance(pkl_data, dict):
                        all_pkl_data.append(pkl_data)
                    else:
                        all_pkl_data.append(pkl_data)

                    print(f"✅ Đã xử lý: {key}")
                except Exception as e:
                    print(f"⚠️ Lỗi đọc file {key}: {e}")
            else:
                print(f"❌ Không thể tải file: {key}")
        
        # ✅ In ra tổng số phần tử đã gộp
        print(f"\n📦 Tổng số đối tượng đã gộp: {len(all_pkl_data)}")
    else:
        print("Không có object_keys nào trong dữ liệu.")
else:
    print("Lỗi khi gọi API:", response.status_code)

✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145536.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145620.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145657.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145708.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145719.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145731.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145744.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145755.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145809.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145821.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145832.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145846.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_data_20250524145859.pkl
✅ Đã xử lý: imcp/encoded-data/2025-05-24/encoded_da

In [16]:
print(len(all_pkl_data))
print(all_pkl_data[0])

2576
{'image_url': 'http://160.191.244.13:9000/lakehouse/imcp/augmented-data/2025-05-24/6e0a007b66b69f7c9aacb9b46606f0ed.jpg', 'pixel_values': tensor([[[[-0.9059, -0.7725, -0.7333,  ...,  0.0275, -0.0039, -0.0118],
          [-0.8196, -0.6471, -0.5843,  ...,  0.0196,  0.0039, -0.0196],
          [-0.6392, -0.6706, -0.6549,  ..., -0.0118, -0.0275, -0.0431],
          ...,
          [-0.2627, -0.2471, -0.2549,  ...,  0.1216,  0.2627,  0.2471],
          [-0.2549, -0.2314, -0.2392,  ...,  0.4275,  0.4745,  0.3804],
          [-0.2549, -0.2314, -0.2235,  ..., -0.0431, -0.0353, -0.0353]],

         [[-0.8902, -0.7490, -0.7098,  ..., -0.3176, -0.3412, -0.3647],
          [-0.8039, -0.6235, -0.5608,  ..., -0.3412, -0.3569, -0.3804],
          [-0.6157, -0.6549, -0.6314,  ..., -0.3569, -0.3647, -0.3882],
          ...,
          [-0.4745, -0.4745, -0.4824,  ..., -0.0667,  0.0745,  0.0745],
          [-0.4824, -0.4824, -0.4980,  ...,  0.2863,  0.3412,  0.2627],
          [-0.4745, -0.4902, -0.4

In [17]:
import random

# Giả sử all_pkl_data là một list chứa toàn bộ dữ liệu
print(f"Tổng số dữ liệu: {len(all_pkl_data)}")  # 2576

# 1. Shuffle để ngẫu nhiên hóa thứ tự (nếu cần)
random.shuffle(all_pkl_data)

# 2. Tính số lượng phần tử cho tập train
train_size = int(0.8 * len(all_pkl_data))  # 80%

# 3. Chia dữ liệu
train_pkl_data = all_pkl_data[:train_size]
test_pkl_data = all_pkl_data[train_size:]

# 4. In kiểm tra
print(f"Số lượng train: {len(train_pkl_data)}")  # ~2060
print(f"Số lượng test: {len(test_pkl_data)}")    # ~516

# ✅ In mẫu một phần tử
print("📄 Mẫu train:")
print(train_pkl_data[0])
print("📄 Mẫu test:")
print(test_pkl_data[0])


Tổng số dữ liệu: 2576
Số lượng train: 2060
Số lượng test: 516
📄 Mẫu train:
{'image_url': 'http://160.191.244.13:9000/lakehouse/imcp/augmented-data/2025-05-24/d62f86bc8aa3b964bb121d956af2f057_0.jpg', 'pixel_values': tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., 

In [20]:
from datetime import datetime
import pickle

print(type(test_pkl_data[0]))

# Lấy ngày hiện tại theo định dạng dd_mm_yy
today_str = datetime.now().strftime("%d_%m_%y")

# Tạo tên file có chứa ngày
test_pkl_filename = f"{today_str}_test_pkl_data.pkl"

# Lưu file
with open(test_pkl_filename, "wb") as f:
    pickle.dump(test_pkl_data, f)

print(f"✅ Đã lưu file test: '{test_pkl_filename}'")


<class 'dict'>
✅ Đã lưu file test: '26_05_25_test_pkl_data.pkl'


## Colected Data

In [ ]:
import pandas as pd

def load_data(data_path="/kaggle/input/traffic-pictures-captioning/augmented/captions_augmented.csv"):
    try:
        df = pd.read_csv(data_path)
        df = df[['original_url','local_path', 'search_query', 'short_caption']].rename(columns={
            'original_url': 'original_url',
            'local_path': 'url',
            'search_query': 'search_query',
            'short_caption': 'caption'
        })
        print(f"Đã tải CSV từ: {data_path} (Kích thước: {df.shape})")
        return df
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại {data_path}")
        raise
    except Exception as e:
        print(f"Lỗi khi đọc CSV: {e}")
        raise

# Load dữ liệu
df = load_data()


# 3. Split data

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

# Hàm chia dữ liệu giữ nguyên
def split_data(df, stratify_col='search_query', test_size=0.1, val_size=0.1, random_state=42):
    unique_urls = df.drop_duplicates('original_url')

    sss_1 = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_val_idx, test_idx in sss_1.split(unique_urls, unique_urls[stratify_col]):
        train_val_urls = unique_urls.iloc[train_val_idx]['original_url']
        test_urls = unique_urls.iloc[test_idx]['original_url']

    df_train_val = df[df['original_url'].isin(train_val_urls)]
    df_test = df[df['original_url'].isin(test_urls)]

    unique_train_val_urls = df_train_val.drop_duplicates('original_url')
    val_ratio = val_size / (1 - test_size)
    sss_2 = StratifiedShuffleSplit(n_splits=1, test_size=val_ratio, random_state=random_state)
    for train_idx, val_idx in sss_2.split(unique_train_val_urls, unique_train_val_urls[stratify_col]):
        train_urls = unique_train_val_urls.iloc[train_idx]['original_url']
        val_urls = unique_train_val_urls.iloc[val_idx]['original_url']

    df_train = df_train_val[df_train_val['original_url'].isin(train_urls)]
    df_val = df_train_val[df_train_val['original_url'].isin(val_urls)]

    return df_train.reset_index(drop=True), df_val.reset_index(drop=True), df_test.reset_index(drop=True)

In [ ]:
def load_split_save(df: pd.DataFrame): 
    # Split
    train_df, val_df, test_df = split_data(df, stratify_col='search_query')
    
    # Kiểm tra
    print("Train size:", len(train_df))
    print("Val size:", len(val_df))
    print("Test size:", len(test_df))
    print(train_df['search_query'].value_counts(normalize=True))
    print(val_df['search_query'].value_counts(normalize=True))
    print(test_df['search_query'].value_counts(normalize=True))

    # Save file
    # Reset index
    train_df = train_df.reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)
    
    # Kiểm tra số lượng mẫu sau khi chia
    print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
    
    # Lưu các DataFrame vào JSON
    train_df.to_json("train.json", orient='records', indent=4, force_ascii=False)
    val_df.to_json("val.json", orient='records', indent=4, force_ascii=False)
    test_df.to_json("test.json", orient='records', indent=4, force_ascii=False)
    
    print("Đã lưu train.json, val.json, test.json")
    return train_df, val_df, test_df, df

# train_df, val_df, test_df, df = load_split_save(df)

# Concat 2 data frame (colected + userAPI)

In [ ]:
from sklearn.model_selection import train_test_split

# Giả sử df là DataFrame chứa cột ảnh hoặc thông tin ảnh
# Bước 1: Chia train và tạm (val + test)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

# Bước 2: Chia tiếp temp thành val và test
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, shuffle=True)

print(f"Số lượng ảnh trong train: {len(train_df)}")
print(f"Số lượng ảnh trong val: {len(val_df)}")
print(f"Số lượng ảnh trong test: {len(test_df)}")

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
df = df.copy().reset_index(drop=True)

# 4. Load feature extractor

In [ ]:
feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

def load_image(local_path, dataset_base_path="/kaggle/input/traffic-pictures-captioning/"):
    """
    Load ảnh từ local_path trong dataset trên Kaggle.
    
    Parameters:
    - local_path (str): Đường dẫn tương đối của ảnh (từ cột 'url' trong DataFrame)
    - dataset_base_path (str): Đường dẫn gốc đến dataset
    
    Returns:
    - image_rgb (numpy.ndarray): Ảnh ở định dạng RGB, hoặc None nếu lỗi
    """
    try:
        # Chuẩn hóa đường dẫn ảnh
        local_path = local_path.lstrip('./')  # Loại bỏ './' nếu có
        full_image_path = os.path.join(dataset_base_path, local_path)
        
        # Đọc ảnh bằng OpenCV
        image = cv2.imread(full_image_path)
        if image is None:
            print(f"Lỗi: Không thể đọc ảnh từ {full_image_path}")
            return None
        
        # Chuyển từ BGR sang RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image_rgb
    
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh {full_image_path}: {e}")
        return None

# 5. Load Tokenizer

In [ ]:
from transformers import AutoTokenizer

# Load tokenizer của BartPho
# tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")

# Kiểm tra vocab size
vocab_size = tokenizer.vocab_size
print(f"VOCAB SIZE: {vocab_size}")

In [ ]:
# In ra các special tokens
print("Special Tokens:", tokenizer.special_tokens_map)

# 6. Các hàm tiền xử lý caption

In [ ]:
def clean_text(text: str) -> str:
    return re.sub(r"[^\w\s,!?.]", "", text).strip()

def to_lowercase(text: str) -> str:
    return text.lower()

def join_vietnamese_compounds(text: str) -> str:
    return ViTokenizer.tokenize(text)

def caption_preprocess(text: str) -> str:
    text = clean_text(text)
    text = to_lowercase(text)
    text = join_vietnamese_compounds(text)
    return text

In [ ]:
text = caption_preprocess("Kiểm tra phân tách từ")

# Tách token
tokens = tokenizer.tokenize(text)

# Chuyển token thành ID
token_ids = tokenizer.convert_tokens_to_ids(tokens)

text = tokenizer.decode(token_ids, skip_special_tokens=True)
# In kết quả
print("List Word (Tokenized):", tokens)
print("List Token ID:", token_ids)
print("Text:", text)

In [ ]:
# Hàm tính số từ của một caption
def count_words(caption):
    preprocessed_caption = caption_preprocess(caption)
    tokens = tokenizer.tokenize(preprocessed_caption)
    return len(tokens)

# Áp dụng hàm cho toàn bộ dataframe
df['word_count'] = df['caption'].apply(count_words)

# Lấy độ dài lớn nhất
max_length = df['word_count'].max()

# In ra
print(f"Độ dài caption dài nhất là: {max_length} từ")

# 7. Tiền xử lý input model

In [ ]:
def process_data(image_url, caption):
    try:
        img_array = load_image(image_url)
        if img_array is None:
            return None
        
        pixel_values = feature_extractor(img_array, return_tensors="pt")["pixel_values"].squeeze(0)
        caption = caption_preprocess(caption)
        tokenized_caption = tokenizer(caption, padding="max_length", max_length=max_length, truncation=True)
        
        return {
            "pixel_values": pixel_values,
            "input_ids": torch.tensor(tokenized_caption["input_ids"]),
            "attention_mask": torch.tensor(tokenized_caption["attention_mask"])
        }
    except Exception as e:
        print(f"Error processing data: {e}")
        return None

# 8. Tạo tập dữ liệu huấn luyện

In [ ]:
class ImageCaptionDataset(Dataset):
    def __init__(self, image_paths, captions):
        self.image_paths = image_paths
        self.captions = captions

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        data = process_data(self.image_paths[idx], self.captions[idx])
        if data is None:
            return self.__getitem__((idx + 1) % len(self))
        return data

def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]
    return {
        "pixel_values": torch.stack([item["pixel_values"] for item in batch]),
        "input_ids": torch.stack([item["input_ids"] for item in batch]),
        "attention_mask": torch.stack([item["attention_mask"] for item in batch])
    }

train_dataset = ImageCaptionDataset(train_df["url"], train_df["caption"])
val_dataset = ImageCaptionDataset(val_df["url"], val_df["caption"])

In [ ]:
print(len(train_pkl_data))
print(train_pkl_data[0])

In [ ]:
print(len(train_dataset))
print(train_dataset[0])

In [ ]:
# Class tạo Dataset từ list các dict
class DatasetFromList(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

# Hàm tạo DatasetFromList từ list dict
def dataset_from_list(data_list):
    return DatasetFromList(data_list)

In [ ]:
new_data = []

for item in train_pkl_data:
    if "image_url" in item:
        del item["image_url"]

    if "pixel_values" in item:
        item["pixel_values"] = item["pixel_values"].squeeze(0)
    
    # Xử lý input_ids và attention_mask: squeeze rồi pad/cắt
    if "input_ids" in item and "attention_mask" in item:
        input_ids = item["input_ids"].squeeze(0)
        attention_mask = item["attention_mask"].squeeze(0)

        # Chuyển tensor sang list để dễ xử lý
        input_ids_list = input_ids.tolist()
        attention_mask_list = attention_mask.tolist()

        # Pad hoặc truncate cho input_ids
        if len(input_ids_list) < max_length:
            pad_length = max_length - len(input_ids_list)
            input_ids_list += [1] * pad_length          # pad với 1 (token id padding)
            attention_mask_list += [0] * pad_length     # pad mask bằng 0
        else:
            input_ids_list = input_ids_list[:max_length]
            attention_mask_list = attention_mask_list[:max_length]

        # Chuyển lại thành tensor
        import torch
        item["input_ids"] = torch.tensor(input_ids_list)
        item["attention_mask"] = torch.tensor(attention_mask_list)

    new_data.append(item)


cleaned_dataset = dataset_from_list(new_data)
print(cleaned_dataset[0])

Ghép dataset cho dữ liệu cũ và dữ liệu mới từ API

In [ ]:
from torch.utils.data import ConcatDataset

train_dataset = ConcatDataset([cleaned_dataset, train_dataset])

In [ ]:
print(len(train_dataset))
print(train_dataset[0])

# 9. Cấu hình LoRA

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
from peft import PeftModel

# Cấu hình LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],  # Các lớp trong GPT2
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# 10. Định nghĩa mô hình

In [ ]:
vit_model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
del vit_model.classifier
vit_model.vit.pooler = torch.nn.Sequential(OrderedDict([
    ('dense', torch.nn.Linear(in_features=768, out_features=768, bias=True)),
    ('activation', torch.nn.Tanh())
]))

config = GPT2Config.from_pretrained("gpt2")
config.add_cross_attention = True
config.vocab_size = vocab_size

gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2", config=config, ignore_mismatched_sizes=True)
gpt2_model.resize_token_embeddings(config.vocab_size)

# Áp dụng LoRA vào GPT2
gpt2_model = get_peft_model(gpt2_model, lora_config)
gpt2_model.print_trainable_parameters()  # kiểm tra số lượng tham số cần huấn luyện

model = VisionEncoderDecoderModel(encoder=vit_model.vit, decoder=gpt2_model)

# 11. Định nghĩa lớp mô hình

In [ ]:
class LoRACaptionWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        super().__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def load_context(self, context):
        try:
            # Đường dẫn đầy đủ đến các thư mục con trong artifact
            model_path = context.artifacts["model_dir"]
            tokenizer_path = os.path.join(model_path, "tokenizer")
            feature_extractor_path = os.path.join(model_path, "feature_extractor")

            
            # Load các thành phần với đường dẫn chính xác
            self.model = VisionEncoderDecoderModel.from_pretrained(model_path).to(self.device)
            self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
            self.feature_extractor = ViTImageProcessor.from_pretrained(feature_extractor_path)
            
            # Đặt model vào chế độ eval
            self.model.eval()
        except Exception as e:
            raise ValueError(f"Error loading model: {str(e)}")

    def predict(self, context, model_input):
        try:
            # Xử lý đầu vào
            if isinstance(model_input, dict):
                image_url = model_input["url"][0] if "url" in model_input else model_input["image_path"][0]
            else:
                image_url = model_input.iloc[0]["url"] if "url" in model_input.columns else model_input.iloc[0]["image_path"]
            
            image = self.load_image(image_url)
            if image is None:
                return [""]
                
            # Tiền xử lý ảnh và tạo caption
            pixel_values = self.feature_extractor(images=image, return_tensors="pt").pixel_values.to(self.device)
            
            output_ids = self.model.generate(
                pixel_values,
                max_length=45,
                min_length=20,
                num_beams=4,
                do_sample=True,
                temperature=0.8,
                top_k=20,
                top_p=0.9,
                no_repeat_ngram_size=3,
                repetition_penalty=2.0,
                early_stopping=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                decoder_start_token_id=self.tokenizer.bos_token_id
            )
            
            caption = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
            return [caption.replace("_", " ").strip()]
            
        except Exception as e:
            print(f"[ERROR] during prediction: {e}")
            return [""]
   
    def load_image(self, image_url):
        try:
            import requests
            import numpy as np
            from PIL import Image
            from io import BytesIO
            
            response = requests.get(image_url, timeout=10)
            if response.status_code != 200:
                return None
                
            image = Image.open(BytesIO(response.content))
            return image.convert("RGB") if image else None
            
        except Exception as e:
            print(f"[ERROR] loading image: {e}")
            return None

# 12. Huấn luyện mô hình

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def evaluate_model(model, val_dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            encoder_outputs = model.encoder(pixel_values=pixel_values)
            outputs = model.decoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                encoder_hidden_states=encoder_outputs.last_hidden_state,
                labels=input_ids
            )
            loss = outputs.loss
            total_loss += loss.item()
    return total_loss / len(val_dataloader)

def train_model(model, train_dataloader, val_dataloader, tokenizer, feature_extractor, lr=5e-5, epochs=1, epoch_count=1, model_name='BartPho_ViT_GPT2_LoRA_ICG', data_part="Part_1", df_log=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Optimizer chỉ update các tham số có requires_grad=True (LoRA adapter)
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    # Thống kê tham số trainable
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())

    run_name = model_name + "_" + data_part

    with mlflow.start_run(run_name=run_name):
        if df_log is not None:
            dataset = mlflow.data.from_pandas(df_log, targets="caption")
            mlflow.log_input(dataset, context="training")

        # Logging các siêu tham số
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("epoch_count", epoch_count)
        mlflow.log_param("data_part", data_part)
        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("train_data_size", len(train_dataloader.dataset))
        mlflow.log_param("val_data_size", len(val_dataloader.dataset))
        mlflow.log_param("trainable_params", trainable_params)
        mlflow.log_param("total_params", total_params)
        mlflow.log_param("trainable_ratio", round(trainable_params / total_params, 4))
        mlflow.log_param("model_name", model_name)

        model.train()
        for epoch in range(epochs):
            total_loss = 0
            for batch in train_dataloader:
                pixel_values = batch["pixel_values"].to(device)
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)

                encoder_outputs = model.encoder(pixel_values=pixel_values)
                outputs = model.decoder(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    encoder_hidden_states=encoder_outputs.last_hidden_state,
                    labels=input_ids
                )
                loss = outputs.loss
                total_loss += loss.item()

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            avg_loss = total_loss / len(train_dataloader)
            val_loss = evaluate_model(model, val_dataloader, device)

            # Log loss theo epoch
            mlflow.log_metric("train_loss", avg_loss, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
            print(f"[Epoch {epoch+1}/{epochs}] Train Loss: {avg_loss:.4f} | Val Loss: {val_loss:.4f}")

            # Lưu mô hình sau mỗi epoch
            epoch_save_path = f"/kaggle/working/{model_name}_epoch_{epoch+1}"
            model.save_pretrained(epoch_save_path)
            tokenizer.save_pretrained(os.path.join(epoch_save_path, "tokenizer"))
            feature_extractor.save_pretrained(os.path.join(epoch_save_path, "feature_extractor"))
            
            # Log toàn bộ thư mục như một artifact
            mlflow.log_artifacts(epoch_save_path, artifact_path=f"epoch_{epoch+1}")

        # Lưu model cuối cùng riêng biệt
        final_model_path = f"/kaggle/working/{model_name}_final"
        model.save_pretrained(final_model_path)
        tokenizer.save_pretrained(os.path.join(final_model_path, "tokenizer"))
        feature_extractor.save_pretrained(os.path.join(final_model_path, "feature_extractor"))
        mlflow.log_artifacts(final_model_path, artifact_path="final_model")

        # Log model dưới dạng pyfunc (nếu cần)
        # Đảm bảo bạn đã định nghĩa class ImageCaptionModel trước

        # Infer signature
        input_example = pd.DataFrame({"url": ["http://example.com/test.jpg"]})
        output_example = pd.DataFrame({"caption": ["Mô tả ảnh"]})
        
        # 2. Tạo model signature
        signature = infer_signature(
            input_example,
            output_example,
            params={"input_types": "string", "output_types": "string"}  # Chỉ định kiểu dữ liệu
        )
        mlflow.pyfunc.log_model(
            artifact_path="model",
            python_model=LoRACaptionWrapper(),
            artifacts={"model_dir": final_model_path},  # Đường dẫn đến thư mục chứa model, tokenizer và feature_extractor
            registered_model_name=model_name,
            signature=signature,
            # input_example=input_example,
            pip_requirements=pip_reqs
        )

In [ ]:
# Tạo k Subset cho train, val
train_subset = Subset(train_dataset, range(len(train_dataset)))  # Lấy train_df.shape[0] dữ liệu đầu tiên từ train_dataset
val_subset = Subset(val_dataset, range(val_df.shape[0]))     # Lấy 4620 dữ liệu đầu tiên từ val_dataset

# Tạo DataLoader cho các subset
train_dataloader = DataLoader(train_subset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)
val_dataloader = DataLoader(val_subset, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)


# 13. Load model pretrain

In [ ]:
# Đường dẫn thư mục chứa các tệp mô hình
model_path = '/kaggle/input/phovit-gptcap/pytorch/default/15'

# Tải mô hình VisionEncoderDecoder
model = VisionEncoderDecoderModel.from_pretrained(model_path, ignore_mismatched_sizes=True)

model = model.to(device)

# 14. Huấn luyện

In [ ]:
from datetime import datetime

# Lấy ngày hiện tại theo format dd-mm-yyyy
today_str = datetime.now().strftime("%d-%m-%Y")
model_name = f'BartPho_ViT_GPT2_LoRA_ICG {today_str} Incremental'

train_model(
    model,
    train_dataloader,
    val_dataloader,
    tokenizer,
    feature_extractor,
    lr=5e-5,
    epochs=2,
    epoch_count=17,
    model_name=model_name,
    data_part="FULL",
    df_log=df
)
